# 1.Prepare All Necessary Data

## 1.1 Necessary package

In [94]:
import pandas as pd 
import numpy as np
from ast import literal_eval
import sklearn
# import sklearn.model_selection
from sklearn.model_selection import train_test_split
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor

## 1.2 Source Data 

In [2]:
# read csv files
# These files are all datasets from Internet without any modified. 
# We need to format them.

# 1.
# @ df_google_map : the places of interests getten from google_map(there are a lot of types here)
df_google_map = pd.read_csv("final_result_from_googleMap.csv")

# df_transaction = pd.read_csv("Transaction/1508-parkomattransaktioner-august-15.csv",encoding='iso-8859-1',engine='python',sep=';', quotechar='"')

# 2.
# @ df_parkomat contains some data about the parking machines' geo information.
df_parkomat =pd.read_csv("parkomat.csv")


# 3.
# @ df_transaction contains all the parking data during the whole 2015.
import glob
path = r'./Transaction' # use your path
all_files = glob.glob(path + "/*.csv")
df_transaction = pd.concat((pd.read_csv(f, sep=';', encoding='iso-8859-1') for f in all_files))
df_transaction['Dato/tid parkeringsstart'] = pd.to_datetime(df_transaction['Dato/tid parkeringsstart'],format='%d-%m-%Y, %H:%M')
df_transaction['Dato/tid parkeringsslut'] = pd.to_datetime(df_transaction['Dato/tid parkeringsslut'],format='%d-%m-%Y, %H:%M')
df_transaction = df_transaction.sort_values(by='Dato/tid parkeringsstart')

,Sættype,Betalingsmåde,Automat,Lokation,Zone,Dato/tid parkeringsstart,Dato/tid parkeringsslut,Beløb,Kreditkortgebyr
0,Betaling,Mønter,3595,COLBJØRNSENSGADE 15,200 - 200 - Grøn del 1,2015-01-01 00:00:00,2015-01-02 08:07:00,"2,00 DKK","0,00 DKK"
1,Betaling,Mønter,1287,NYROPSGADE 27,200 - 200 - Grøn del 1,2015-01-01 00:01:00,2015-01-02 08:17:00,"5,00 DKK","0,00 DKK"
2,Betaling,Mønter,1288,NaN,? - ?,2015-01-01 00:01:00,2015-01-02 08:03:00,"1,00 DKK","0,00 DKK"
3,Betaling,VISA Dankort,2442,WILLEMOESGADE 66,315 - 315 - Blå del 16,2015-01-01 00:01:00,2015-01-02 12:05:00,"45,00 DKK","0,50 DKK"
4,Betaling,Mønter,2184,POLITITORVET 14,200 - 200 - Grøn del 1,2015-01-01 00:04:00,2015-01-02 08:03:00,"1,00 DKK","0,00 DKK"


## 1.3 Format these data
### 1.3.1 Get All Types of Google_Map's Locations

In [3]:
# get all the type names
# get all the type names from google_map
location_type = set([])
for i in df_google_map['type']:
    mlist = literal_eval(i)
    for item in mlist:
        location_type.add(item)


# add these types to df (similar to one-hot code)
for i in location_type:
    df_parkomat.insert(len(df_parkomat.columns),i,0)
    df_google_map.insert(len(df_google_map.columns),i,0)

for i in range(len(df_google_map)):
    for type_now in literal_eval(df_google_map.loc[i,'type']):
        df_google_map.loc[i,type_now]=1


### The types of locations in df_google_map (`location_type`) are following:
> {'accounting',
 'airport',
 'amusement_park',
 'aquarium',
 'art_gallery',
 'atm',
 'bakery',
 'bank',
 'bar',
 'beauty_salon',
 'bicycle_store',
 'book_store',
 'bowling_alley',
 'bus_station',
 'cafe',
 'car_dealer',
 'car_rental',
 'car_repair',
 'car_wash',
 'casino',
 'cemetery',
 'church',
 'city_hall',
 'clothing_store',
 'convenience_store',
 'courthouse',
 'dentist',
 'department_store',
 'doctor',
 'drugstore',
 'electrician',
 'electronics_store',
 'embassy',
 'establishment',
 'finance',
 'fire_station',
 'florist',
 'food',
 'funeral_home',
 'furniture_store',
 'gas_station',
 'general_contractor',
 'grocery_or_supermarket',
 'gym',
 'hair_care',
 'hardware_store',
 'health',
 'home_goods_store',
 'hospital',
 'insurance_agency',
 'jewelry_store',
 'laundry',
 'lawyer',
 'library',
 'liquor_store',
 'local_government_office',
 'locality',
 'locksmith',
 'lodging',
 'meal_delivery',
 'meal_takeaway',
 'mosque',
 'movie_rental',
 'movie_theater',
 'moving_company',
 'museum',
 'natural_feature',
 'night_club',
 'painter',
 'park',
 'parking',
 'pet_store',
 'pharmacy',
 'physiotherapist',
 'place_of_worship',
 'plumber',
 'point_of_interest',
 'police',
 'political',
 'post_office',
 'premise',
 'primary_school',
 'real_estate_agency',
 'restaurant',
 'roofing_contractor',
 'rv_park',
 'school',
 'secondary_school',
 'shoe_store',
 'shopping_mall',
 'spa',
 'stadium',
 'storage',
 'store',
 'sublocality',
 'sublocality_level_1',
 'sublocality_level_2',
 'subway_station',
 'supermarket',
 'tourist_attraction',
 'train_station',
 'transit_station',
 'travel_agency',
 'university',
 'veterinary_care',
 'zoo'}

### 1.3.2 Explore The Basic Info of These 3 Datasets

First datasets `df_transaction`, it contains the :
* 'paying methods'(which does matter in this project)
* 'Vending machine ID'(could be used to connect the `df_transaction` and `df_parkomat` to get the concrete address of this machine) 
* 'Date and hour' is also a very useful info in this datasets, I want to find out the distribution of hour per parking point and the regulation of the time series.
* 'cost' is relatively not that important here

In [4]:
df_transaction.sort_values(by=['Automat','Dato/tid parkeringsstart','Dato/tid parkeringsslut']).head()

,Sættype,Betalingsmåde,Automat,Lokation,Zone,Dato/tid parkeringsstart,Dato/tid parkeringsslut,Beløb,Kreditkortgebyr
676,Betaling,Mønter,152,NaN,? - ?,2015-01-01 18:05:00,2015-01-02 08:02:00,"1,00 DKK","0,00 DKK"
1130,Betaling,Mønter,152,NaN,? - ?,2015-01-01 23:41:00,2015-01-02 08:02:00,"1,00 DKK","0,00 DKK"
1283,Betaling,Mønter,152,NaN,? - ?,2015-01-02 05:18:00,2015-01-02 12:08:00,"124,00 DKK","0,00 DKK"
2967,Betaling,VISA Dankort,152,NaN,? - ?,2015-01-02 09:35:00,2015-01-02 10:45:00,"35,00 DKK","0,50 DKK"
3002,Betaling,Mønter,152,NaN,? - ?,2015-01-02 09:36:00,2015-01-02 14:56:00,"160,00 DKK","0,00 DKK"


Second datasets `df_parkomat`, we would  use the 'parkomat_id' to connect it with `df_transaction`.
There are a lot of columns of location types added above. They are used to calculate how much points are within a radius(which could be set by hand)

In [5]:

df_parkomat.head()

,FID,id,parkomat_id,type,vejkode,p_zone,beboer_zone,vejnavn,husnr,placering,...,beauty_salon,shopping_mall,meal_delivery,car_rental,mosque,shoe_store,atm,funeral_home,lodging,cafe
0,parkomat.1,467,152,P-info,1396.0,Rød,IB,Dyrkøb,5,NaN,...,0,0,0,0,0,0,0,0,0,0
1,parkomat.2,466,153,Betalingsautomat,1396.0,Rød,IB,Dyrkøb,OF/3,NaN,...,0,0,0,0,0,0,0,0,0,0
2,parkomat.3,470,156,Betalingsautomat,148.0,Rød,IB,H.C. Andersens Boulevard,9,Eksisterende fortovsø,...,0,0,0,0,0,0,0,0,0,0
3,parkomat.4,471,157,Betalingsautomat,3348.0,Rød,IB,Jarmers Plads,7,Eksisterende fortovsø,...,0,0,0,0,0,0,0,0,0,0
4,parkomat.5,472,158,P-info,5220.0,Rød,IB,Nørre Voldgade,OF/6,NaN,...,0,0,0,0,0,0,0,0,0,0


In [6]:
df_google_map.head()

,name,lat,lng,type,insurance_agency,travel_agency,electrician,police,car_repair,dentist,...,beauty_salon,shopping_mall,meal_delivery,car_rental,mosque,shoe_store,atm,funeral_home,lodging,cafe
0,Copenhagen,55.676097,12.568337,"['locality', 'political']",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,VIK Rengøring,55.708810,12.569768,"['point_of_interest', 'establishment']",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,IP-Support.dk Computerværksted,55.708099,12.573003,"['point_of_interest', 'establishment']",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Vintage 199,55.708056,12.570833,"['point_of_interest', 'clothing_store', 'store...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Kan Rep Låse,55.708339,12.569962,"['locksmith', 'point_of_interest', 'establishm...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 1.3.3 Explore more basic info in these datasets

Firstly I explore the dataset df_google_map.There are 21470 rows of places data. But what matters is every place would have multi tags.
According to the summary of df_google_map,the number of 'establishment' is 21454 and the number of 'point_of_interest' is also 21454. 

What's more, there are also tags like 'bus station','subway_station' and 'transit station'. Some 'railway station' would result in the rise of parking counts but the 'bus station' and 'subway_station' would result in the decrease of that. It is a meaningful point which we could study with our data.



In [7]:
df_google_map_points_count = df_google_map[location_type].apply(sum)
print(df_google_map_points_count)
# print(len(df_google_map))

insurance_agency     33
travel_agency       115
electrician          64
police                9
car_repair           69
                   ... 
shoe_store           37
atm                  21
funeral_home         17
lodging             177
cafe                206
Length: 106, dtype: int64


### 1.3.3 Calculate the number of various types of points within a certain range 

#### Firstly, the function of calculate the distance with lat and lng.

In [8]:
# now try to counting the number of different type points 
# copenhagen : lat:55.6761° N,  lng: 12.5683° E
from math import radians, cos, sin, asin, sqrt
def geodistance(lng1,lat1,lng2,lat2):
    lng1, lat1, lng2, lat2 = map(radians, [lng1, lat1, lng2, lat2])
    dlon=lng2-lng1
    dlat=lat2-lat1
    a=sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2 
    dis=2*asin(sqrt(a))*6371*1000
    return dis


RADIUS = 300
## =========================================================BEGINNING==========================================================
## the following codes are used to calculate the amount of different types of location within a concrete raidus here
## it takes too long time to run, so I saved the result as a csv file. Only need to read the csv file we could get the result
#
#
# df_parkomat_count = df_parkomat.copy()
# # df_parkomat_count.insert(0,'Count_radius_300m',0)
# list_column_parkomat_count = location_type.copy()
# list_column_parkomat_count.add('wkb_geometry')
# list_column_parkomat_count.add('['parkomat_id']')
# df_parkomat_count = df_parkomat_count[list_column_parkomat_count]
# # df_parkomat_count['wkb_geometry']
# df_parkomat_count['lng']  = df_parkomat_count['wkb_geometry'].map(lambda x:(re.split('POINT \(| |\)',x)[1]))
# df_parkomat_count['lat']  = df_parkomat_count['wkb_geometry'].map(lambda x:(re.split('POINT \(| |\)',x)[2]))


# for j in range(len(df_parkomat_count)):
#     print(j)
#     list_index_find = []
#     for i in range(len(df_google_map)):
#         distance = geodistance(df_google_map[i:i+1]['lng'],df_google_map[i:i+1]['lat'],df_parkomat_count[j:j+1]['lng'],df_parkomat_count[j:j+1]['lat'])
#         if(distance <= RADIUS):
#             list_index_find.append(i)
#     df_parkomat_count.loc[j,location_type]=df_google_map.loc[list_index_find,location_type].sum()
#     if(j%100==0):
#         df_parkomat_count.to_csv('df_parkomat_count_300m.csv')

# df_parkomat_count.to_csv('df_parkomat_count_300m.csv')

## =============================================================END===============================================================
df_parkomat_count = pd.read_csv("data_generated_by_wenjie/df_parkomat_count_300m.csv",index_col=0)



Now the `df_parkmot_count` contains the count of every type within a certain radius(300m)

In [10]:
df_parkomat_count.head()

,book_store,supermarket,church,hair_care,shoe_store,bicycle_store,local_government_office,primary_school,id,sublocality_level_2,...,gym,car_wash,electrician,beauty_salon,drugstore,aquarium,police,roofing_contractor,lng,lat
0,3,0,2,1,1,1,0,0,467,0,...,0,0,0,0,0,0,0,0,12.572666,55.679003
1,4,0,2,1,1,1,0,0,466,0,...,0,0,0,0,0,0,0,0,12.572937,55.679216
2,1,1,0,1,2,1,3,0,470,0,...,2,0,0,2,0,0,0,0,12.566108,55.677827
3,1,1,0,1,2,1,4,0,471,0,...,2,0,0,4,0,0,0,0,12.565577,55.678517
4,1,0,1,1,2,1,4,0,472,0,...,2,0,0,4,0,0,0,0,12.566308,55.679171


### 1.3.4 Working with the `df_transaction`
`df_transaction` is really an important dataset.


In [11]:
df_transaction_temp = df_transaction.sort_values(by=['Automat','Dato/tid parkeringsstart','Dato/tid parkeringsslut'])
# a = t['Automat'].drop_duplicates()
# b= df_parkomat['parkomat_id'].drop_duplicates()
# k = pd.Series(list(set(a).intersection(set(b))))
# k.sort_values()



In [12]:
df_transaction_now_using = df_transaction_temp[(df_transaction_temp['Dato/tid parkeringsslut']-df_transaction_temp['Dato/tid parkeringsstart'])>=pd.Timedelta(0,'s')]
df_transaction_now_using.loc[:,'diff_time'] = (df_transaction_now_using['Dato/tid parkeringsslut'] - df_transaction_now_using['Dato/tid parkeringsstart']).dt.seconds/60
df_transaction_now_using.loc[:,'start_hour'] = df_transaction_now_using['Dato/tid parkeringsstart'].dt.hour
df_transaction_now_using.loc[:,'leave_hour'] = df_transaction_now_using['Dato/tid parkeringsslut'].dt.hour

df_transaction_now_using.head()

D:\Development_tool\anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
D:\Development_tool\anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


,Sættype,Betalingsmåde,Automat,Lokation,Zone,Dato/tid parkeringsstart,Dato/tid parkeringsslut,Beløb,Kreditkortgebyr,diff_time,start_hour,leave_hour
676,Betaling,Mønter,152,NaN,? - ?,2015-01-01 18:05:00,2015-01-02 08:02:00,"1,00 DKK","0,00 DKK",837.0,18,8
1130,Betaling,Mønter,152,NaN,? - ?,2015-01-01 23:41:00,2015-01-02 08:02:00,"1,00 DKK","0,00 DKK",501.0,23,8
1283,Betaling,Mønter,152,NaN,? - ?,2015-01-02 05:18:00,2015-01-02 12:08:00,"124,00 DKK","0,00 DKK",410.0,5,12
2967,Betaling,VISA Dankort,152,NaN,? - ?,2015-01-02 09:35:00,2015-01-02 10:45:00,"35,00 DKK","0,50 DKK",70.0,9,10
3002,Betaling,Mønter,152,NaN,? - ?,2015-01-02 09:36:00,2015-01-02 14:56:00,"160,00 DKK","0,00 DKK",320.0,9,14


In [117]:
df_transaction_start_hour_count = df_transaction_now_using.groupby(['start_hour','Automat']).size().reset_index(name='counts')
df_transaction_start_hour_count = df_transaction_start_hour_count.rename(columns={"Automat":"parkomat_id"})


df_transaction_leave_hour_count = df_transaction_now_using.groupby(['leave_hour','Automat']).size().reset_index(name='counts')
df_transaction_leave_hour_count = df_transaction_leave_hour_count.rename(columns={"Automat":"parkomat_id"})

# df_transaction_start_hour_count[df_transaction_start_hour_count['start_hour']==0]


,start_hour,parkomat_id,counts
0,0,152,248
1,0,153,285
2,0,156,346
3,0,157,194
4,0,158,74
...,...,...,...
1446,0,9621,6
1447,0,9622,2
1448,0,9623,3
1449,0,9624,4


In [112]:
def randomForest(X,y,X_test,y_test):

# Instantiate model with 1000 decision trees
    rf = RandomForestRegressor(n_estimators = 300, random_state = 40)
# Train the model on training data
    rf.fit(X, y)
#test 
    predictions = rf.predict(X_test)
# Calculate the absolute errors
    errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
    print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')
# Calculate mean absolute percentage error (MAPE)
    mape = 100 * (errors / y_test)
# Calculate and display accuracy
    accuracy = 100 - np.mean(mape)
    print('Accuracy:', round(accuracy, 2), '%.')
    return errors
# ###############################################################


In [113]:
# for i in range(24):
#     df_now_using = df_transaction_start_hour_count[df_transaction_start_hour_count['start_hour']==(i)]
#     df_now_using = df_now_using.merge(df_parkomat_count,how='inner',on = 'parkomat_id')
#     train = df_now_using.drop(
#         columns=['start_hour',
#         'parkomat_id',
#         'id',
#         'counts',
#         'lng',
#         'lat',
#         'sublocality_level_2',
#         'sublocality_level_1',
#         'sublocality',
#         'establishment',
#         'point_of_interest',
#         'wkb_geometry'])
#     target = df_now_using['counts']
#     X,X_test, y, y_test = sklearn.model_selection.train_test_split(train,target,train_size = int(len(train)*0.9),random_state = 0)
#     print("start:",i)
#     randomForest(X,y,X_test,y_test)



start: 0
Mean Absolute Error: 12.57 degrees.
Accuracy: -51.96 %.
start: 1
Mean Absolute Error: 10.56 degrees.
Accuracy: -67.07 %.
start: 2
Mean Absolute Error: 4.5 degrees.
Accuracy: -37.92 %.
start: 3
Mean Absolute Error: 3.73 degrees.
Accuracy: -38.84 %.
start: 4
Mean Absolute Error: 4.16 degrees.
Accuracy: -56.1 %.
start: 5
Mean Absolute Error: 13.06 degrees.
Accuracy: -233.26 %.
start: 6
Mean Absolute Error: 43.38 degrees.
Accuracy: -107.55 %.
start: 7
Mean Absolute Error: 76.07 degrees.
Accuracy: -56.79 %.
start: 8
Mean Absolute Error: 142.61 degrees.
Accuracy: -54.49 %.
start: 9
Mean Absolute Error: 154.52 degrees.
Accuracy: -69.22 %.
start: 10
Mean Absolute Error: 182.84 degrees.
Accuracy: -37.28 %.
start: 11
Mean Absolute Error: 192.35 degrees.
Accuracy: -176.16 %.
start: 12
Mean Absolute Error: 197.72 degrees.
Accuracy: -33.17 %.
start: 13
Mean Absolute Error: 200.04 degrees.
Accuracy: 3.35 %.
start: 14
Mean Absolute Error: 219.75 degrees.
Accuracy: -42.17 %.
start: 15
Mean Ab

In [114]:
# for i in range(24):
#     df_now_using = df_transaction_leave_hour_count[df_transaction_leave_hour_count['leave_hour']==(i)]
#     df_now_using = df_now_using.merge(df_parkomat_count,how='inner',on = 'parkomat_id')
#     train = df_now_using.drop(
#         columns=['leave_hour',
#         'parkomat_id',
#         'id',
#         'counts',
#         'lng',
#         'lat',
#         'sublocality_level_2',
#         'sublocality_level_1',
#         'sublocality',
#         'establishment',
#         'point_of_interest',
#         'wkb_geometry'])
#     target = df_now_using['counts']
#     X,X_test, y, y_test = sklearn.model_selection.train_test_split(train,target,train_size = int(len(train)*0.9),random_state = 0)
#     print("start:",i)
#     randomForest(X,y,X_test,y_test)


start: 0
Mean Absolute Error: 43.23 degrees.
Accuracy: -5.99 %.
start: 1
Mean Absolute Error: 29.39 degrees.
Accuracy: -62.7 %.
start: 2
Mean Absolute Error: 11.36 degrees.
Accuracy: -28.6 %.
start: 3
Mean Absolute Error: 10.92 degrees.
Accuracy: -44.88 %.
start: 4
Mean Absolute Error: 7.76 degrees.
Accuracy: -8.81 %.
start: 5
Mean Absolute Error: 10.5 degrees.
Accuracy: -19.06 %.
start: 6
Mean Absolute Error: 11.71 degrees.
Accuracy: -30.06 %.
start: 7
Mean Absolute Error: 18.3 degrees.
Accuracy: -15.02 %.
start: 8
Mean Absolute Error: 156.38 degrees.
Accuracy: 10.65 %.
start: 9
Mean Absolute Error: 104.31 degrees.
Accuracy: -50.43 %.
start: 10
Mean Absolute Error: 109.2 degrees.
Accuracy: -12.56 %.
start: 11
Mean Absolute Error: 156.42 degrees.
Accuracy: -63.13 %.
start: 12
Mean Absolute Error: 175.31 degrees.
Accuracy: -31.27 %.
start: 13
Mean Absolute Error: 178.74 degrees.
Accuracy: -50.41 %.
start: 14
Mean Absolute Error: 195.08 degrees.
Accuracy: -13.82 %.
start: 15
Mean Absolut

In [13]:
## =========DRAWING MAPS===================================
# # # test = literal_eval(df_parking[0:1]['wkb_geometry'])
# # t = (df_parking.loc[6581,['wkb_geometry']])['wkb_geometry']
# # point_around = df_google_map
# # point_around = point_around.drop(index=point_around.index)
# # for point1 in t:
# #     for point2 in df_google_map:
# # len(df_parkomat)
# #1602
# df_parkomat.head()
# df_parkomat['wkb_geometry'][1]
# # df_parkomat['lat'] = df_parkomat['wkb_geometry'].apply(lambda x: x.)
# import re
# t = re.split('POINT \(| |\)',df_parkomat['wkb_geometry'][1])[1:3]

# def strpoint_to_list(str):
#     temp_list=[]

# df_parking = pd.read_csv("taelling_p_pladser.csv", parse_dates=['aar_mdr'])
# df_parking = df_parking[(df_parking['aar_mdr']>='2012') & (df_parking['aar_mdr']<='2021')]
# df_parking = df_parking.sort_values(by='aar_mdr')
# # df_parking = df_parking[df_parking.aar_mdr != '201  405']
# df_parking['aar_mdr'] = df_parking['aar_mdr'].apply(lambda x: str(x)).apply(lambda x : x[:4]+'/'+x[4:])
# def format_geo(x):
#     x = str(x)
#     for i in ")(,":
#         x = x.replace(i,'')
#     x = x.split()[1:]
#     locations = []
#     for i in range(0,len(x),2):
#         locations.append(x[i:i+2])
#     return locations
# df_parking['wkb_geometry'] = df_parking['wkb_geometry'].apply(format_geo)

In [14]:
# import folium
# cp_map = folium.Map(location=[55.6760968, 12.5683371], zoom_start=13)
# for i in range(len(df_google_map)):
#     folium.Circle(
#         radius=1,
#         location=[df_google_map.loc[i,'lat'], df_google_map.loc[i,'lng']],
#         color="blue",
#         fill=True
#     ).add_to(cp_map)

# for i in range(len(df_parkomat)):
#     t = re.split('POINT \(| |\)',df_parkomat['wkb_geometry'][i])[1:3]
#     # folium.Circle(
#     #     radius=3,
#     #     location=[t[1], t[0]],
#     #     color="green",
#     #     fill=True
#     # ).add_to(cp_map)
#     folium.Marker(
#     location=[t[1], t[0]],
#     icon=folium.Icon(color="green")
# ).add_to(cp_map)



# df_201703 = df_parking[df_parking['aar_mdr'] == '2017/03']
# df_201703 = df_201703.dropna(subset=['wkb_geometry'])
# df_201703 = df_201703[df_201703['wkb_geometry'].apply(lambda x: len(x)) > 0]
# df_201703 = df_201703[df_201703['lovlig_p_kl_12']!=0]
# df_201703 = df_201703[df_201703['lovlig_p_kl_17']!=0]
# df_201703 = df_201703[df_201703['lovlig_p_kl_22']!=0]
# df_201703 = df_201703.drop_duplicates(subset=['taelle_id'], keep='first')

# import colorsys

# for i, row in df_201703.iterrows():
#     points = []
#     if len(row['wkb_geometry']) == 0:
#         continue
#     rate = int(row['belaegning_kl_12_pct'])
#     if rate > 100:
#         rate = 100
#     rgb = colorsys.hsv_to_rgb((100 -rate)/ 300., 1.0, 1.0)
#     color = '#%02x%02x%02x' % tuple([round(255*x) for x in rgb])
#     for location in row['wkb_geometry']:
#         point = []
#         point.append(float(location[1]))
#         point.append(float(location[0]))
#         points.append(point)
#     popup = row['vejnavn'] + '-' + str(rate) + '%'
#     folium.PolyLine(points, color=color, popup=popup).add_to(cp_map)


# cp_map

In [209]:
## This sector is used to calculate how many cars parking in every park per hour.
## ============================================BEGIN===========================
# time = pd.to_datetime("2015-01-01 00:00:00")
# df_transaction_hour_series =pd.DataFrame({'Automat':[],'counts':[],'datetime':[]})
# while time<pd.to_datetime("2016-01-01 00:00:00"):
#     temp = df_transaction[(df_transaction['Dato/tid parkeringsstart']<=time+pd.Timedelta(1,'h'))&(df_transaction['Dato/tid parkeringsslut']>=time)].groupby(['Automat']).size().reset_index(name='counts')
#     temp['datetime'] = time
#     time = time + pd.Timedelta(1,'h')
#     df_transaction_hour_series = df_transaction_hour_series.append(temp)
#     print(time)
## ====================================END=================================
df_transaction_hour_series = pd.read_csv("data_generated_by_wenjie/df_transaction_hour_series.csv",usecols=['Automat','counts','datetime'],parse_dates=['datetime'])

In [212]:
# df_transaction_hour_series.to_csv('df_transaction_hour_series.csv')
series_max_parking_count = df_transaction_hour_series.groupby(['Automat']).max()['counts']
# df_transaction_hour_series[(df_transaction_hour_series['Automat']==153)]

In [213]:
series_max_parking_count = 

Automat
152.0      38.0
153.0      61.0
156.0      71.0
157.0      49.0
158.0      31.0
           ... 
9624.0     16.0
9625.0     10.0
69691.0    29.0
69692.0    32.0
69693.0    33.0
Name: counts, Length: 1522, dtype: float64